### For Graduate QM Term Project 2023
Geraldine (gera0037@ntu.edu.sg).

# Quantum Cryptography – BB84

This Jupyter notebook is an appendix / supplementary to the report. Results have been included in the report, but this is a more detailed documentation of my computational process.

Download the ipynb file from my <a href="https://github.com/gravitance/school/blob/main/coursework/term%20project%20-%20BB84.ipynb">Github</a> or <a href="https://drive.google.com/drive/folders/1_qUJzKGbcYd97tS0bAS1vBVDWMqT5NLL?usp=share_link">Gdrive</a> to run the simulations or interact with the code.

Brief contents:
- Working through the BB84 protocol in matrix and state representation
- Simulating the BB84 protocol for:
    - Case without eavesdropper
    - Case with eavesdropper

## Before attempting the simulation,

I try to first visualize and understand how the protocol works mathematically, using matrix representation and states.

In [1]:
import numpy as np
import sympy as sp
import random

### states, encoding bases.

In [2]:
bit0   = np.array([1.0, 0.0])
bit1 = np.array([0.0, 1.0])

h   = np.array([1.0, 0.0])
v = np.array([0.0, 1.0])
d = 1/np.sqrt(2) * (h - v)
g = 1/np.sqrt(2) * (h + v)

### all possible encoded states.

In [3]:
bit0_h = np.kron(bit0,h)
bit0_v = np.kron(bit0,v)
bit0_d = np.kron(bit0,d)
bit0_g = np.kron(bit0,g)

bit1_h = np.kron(bit1,h)
bit1_v = np.kron(bit1,v)
bit1_d = np.kron(bit1,d)
bit1_g = np.kron(bit1,g)

"""
display(sp.Matrix(bit0_h))
display(sp.Matrix(bit0_v))
display(sp.Matrix(bit0_d))
display(sp.Matrix(bit0_g))

display(sp.Matrix(bit1_h))
display(sp.Matrix(bit1_v))
display(sp.Matrix(bit1_d))
display(sp.Matrix(bit1_g))
"""

'\ndisplay(sp.Matrix(bit0_h))\ndisplay(sp.Matrix(bit0_v))\ndisplay(sp.Matrix(bit0_d))\ndisplay(sp.Matrix(bit0_g))\n\ndisplay(sp.Matrix(bit1_h))\ndisplay(sp.Matrix(bit1_v))\ndisplay(sp.Matrix(bit1_d))\ndisplay(sp.Matrix(bit1_g))\n'

### matrix operators for components.

In [4]:
comp = np.kron(np.outer(h,h.conj()), np.eye(2)) + np.kron(np.outer(v,v.conj()),np.eye(2))
#display(sp.Matrix(comp))

had = np.kron(np.outer(d,d.conj()), np.eye(2)) + np.kron(np.outer(g,g.conj()),np.eye(2))
#display(sp.Matrix(had))

mirror = np.array([[0,1],[1,0]])
rotator = 1/np.sqrt(2) * np.array([[1,-1],[1,1]])
mrot = np.kron(mirror,rotator)
#display(sp.Matrix(mrot))

pbs = np.array([[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]])

pmes = np.dot(pbs,mrot)
display(sp.Matrix(pmes))

Matrix([
[                0,                  0, 0.707106781186547, -0.707106781186547],
[0.707106781186547,  0.707106781186547,                 0,                  0],
[0.707106781186547, -0.707106781186547,                 0,                  0],
[                0,                  0, 0.707106781186547,  0.707106781186547]])

### function to measure and output final state.
to make my life easier. basically it shoots alice's encoded bit over to bob and measures it with the chosen basis.

In [5]:
def shoot(init,basis):
    states = {'bit0_h': bit0_h, 'bit0_v' : bit0_v, 'bit0_d' : bit0_d, 'bit0_g' : bit0_g, 'bit1_h' : bit1_h, 'bit1_v' : bit1_v, 'bit1_d' : bit1_d, 'bit1_g' : bit1_g}
    
    for key in states:
        if (states[key] == init).all():
            initial = key
    
    psi0 = init
    print(f"initial state: {initial}")
    print("before measurement:")
    display(sp.Matrix(psi0))
    
    if basis == "comp":
        ms = np.dot(comp,np.dot(pbs,psi0))

    elif basis == "had":
        ms = np.dot(had,np.dot(pmes,psi0))

    else:
        print("error.")
        return
    
    print(f"measuring basis: {basis}")
    print("after measurement:")
    display(sp.Matrix(ms))
        
    ph = abs(ms[0])**2
    pv = abs(ms[1])**2
    pd = abs(ms[2])**2
    pg = abs(ms[3])**2

    print(f"prob 0_h: {ph}\nprob 0_v: {pv}")
    print(f"prob 1_h: {pd}\nprob 1_v: {pg}")

    states.clear()
    

### measure in computational basis.

In [6]:
print("--- INITIAL ENCODED H STATE ---\n")

init = bit0_h
basis = "comp"
shoot(init,basis)

print("-----")

init = bit1_h
basis = "comp"
shoot(init,basis)

print()

print("--- INITIAL ENCODED V STATE ---\n")

init = bit0_v
basis = "comp"
shoot(init,basis)

print("-----")

init = bit1_v
basis = "comp"
shoot(init,basis)

print()

print("--- INITIAL ENCODED D STATE ---\n")

init = bit0_d
basis = "comp"
shoot(init,basis)

print("-----")

init = bit1_d
basis = "comp"
shoot(init,basis)

print()

print("--- INITIAL ENCODED G STATE ---\n")

init = bit0_g
basis = "comp"
shoot(init,basis)

print("-----")

init = bit1_g
basis = "comp"
shoot(init,basis)

--- INITIAL ENCODED H STATE ---

initial state: bit0_h
before measurement:


Matrix([
[1.0],
[  0],
[  0],
[  0]])

measuring basis: comp
after measurement:


Matrix([
[1.0],
[  0],
[  0],
[  0]])

prob 0_h: 1.0
prob 0_v: 0.0
prob 1_h: 0.0
prob 1_v: 0.0
-----
initial state: bit1_h
before measurement:


...

prob 0_h: 0.4999999999999999
prob 0_v: 0.0
prob 1_h: 0.0
prob 1_v: 0.4999999999999999
-----
initial state: bit1_g
before measurement:


Matrix([
[                0],
[                0],
[0.707106781186547],
[0.707106781186547]])

measuring basis: comp
after measurement:


Matrix([
[                0],
[0.707106781186547],
[0.707106781186547],
[                0]])

prob 0_h: 0.0
prob 0_v: 0.4999999999999999
prob 1_h: 0.4999999999999999
prob 1_v: 0.0


### measure in hadamard basis.

In [7]:
print("--- INITIAL ENCODED H STATE ---\n")

init = bit0_h
basis = "had"
shoot(init,basis)

print("-----")

init = bit1_h
basis = "had"
shoot(init,basis)

print()

print("--- INITIAL ENCODED V STATE ---\n")

init = bit0_v
basis = "had"
shoot(init,basis)

print("-----")

init = bit1_v
basis = "had"
shoot(init,basis)

print()

print("--- INITIAL ENCODED D STATE ---\n")

init = bit0_d
basis = "had"
shoot(init,basis)

print("-----")

init = bit1_d
basis = "had"
shoot(init,basis)

print()

print("--- INITIAL ENCODED G STATE ---\n")

init = bit0_g
basis = "had"
shoot(init,basis)

print("-----")

init = bit1_g
basis = "had"
shoot(init,basis)

--- INITIAL ENCODED H STATE ---

initial state: bit0_h
before measurement:


Matrix([
[1.0],
[  0],
[  0],
[  0]])

measuring basis: had
after measurement:


Matrix([
[                0],
[0.707106781186547],
[0.707106781186547],
[                0]])

prob 0_h: 0.0
prob 0_v: 0.4999999999999998
prob 1_h: 0.4999999999999998
prob 1_v: 0.0
-----
initial state: bit1_h
before measurement:


...

prob 0_h: 0.0
prob 0_v: 0.9999999999999991
prob 1_h: 0.0
prob 1_v: 0.0
-----
initial state: bit1_g
before measurement:


Matrix([
[                0],
[                0],
[0.707106781186547],
[0.707106781186547]])

measuring basis: had
after measurement:


Matrix([
[  0],
[  0],
[  0],
[1.0]])

prob 0_h: 0.0
prob 0_v: 0.0
prob 1_h: 0.0
prob 1_v: 0.9999999999999991


#### Summary:
- When encoded bit is in computational basis (H or V):
    - Measured in computational basis --> get same polarization (H or V) with probability 1
    - Measured in hadamard basis --> get H and V with equal probability of $\frac{1}{2}$
- When encoded bit is in hadamard basis (DL or DR):
    - Measured in computational basis after rotation --> get H and V with equal probability of $\frac{1}{2}$
    - Measured in hadamard basis after rotation --> get same polarization (H for DL, or V for DR) with probability 1

## Preliminaries – Defining functions used.

- `encode(bitlist)`
    - takes in array of bits (0s and 1s)
    - encodes each bit in random basis
    - returns array of encoded bits and array of encoding bases used
- `decode(bitlist)`
    - takes in array of encoded bits (h, v, dl, dr)
    - decodes each bit in random basis
    - returns array of decoded bits and array of decoding bases used
- `match(ebase, dbase)`
    - takes in arrays of encoding bases and decoding bases
    - checks each basis
    - returns array of indexes where bases match, match count, arrays of encoded and decoded bits at indexes of matching bases
- `compare(ematch, dmatch, n)`
    - takes in arrays of matched encoded and decoded bits, number n
    - checks n number of bits from each array at the same index
    - computes error rate
    - returns arrays of matched decoded and encoded bits with checked bits discarded, and error rate

In [8]:
def encode(bitlist):
    
    encoded = []
    ebase = []

    for bit in bitlist:
        
        base = int(np.random.choice([0,1]))
        ebase.append(base)

        if base == 0:   # computational basis +
            if bit == 0:   # horizontal
                ebit = "h"
            else:          # vertical
                ebit = "v"

        else:           # hadamard basis X
            if bit == 0:   # diagonal left
                ebit = "dl"
            else:          # diagonal right
                ebit = "dr"
        
        encoded.append(ebit)
        
    return encoded, ebase

In [9]:
def decode(bitlist):
    
    decoded = []
    dbase = []
    
    for bit in bitlist:
    
        base = int(np.random.choice([0,1]))
        dbase.append(base)
    
        if base == 0:   # computational basis +
            if bit == "h" or bit == "v":   # encode basis = decode basis
                dbit = bit
            else:                          # encode basis != decode basis
                
                # randomly decide on v or h in computational basis
                randnum = int(np.random.choice([0,1]))
                if randnum == 0:
                    dbit = "h"
                else:
                    dbit = "v"

        else:           # hadamard basis X
            if bit == "dl" or bit == "dr":   # encode basis = decode basis
                dbit = bit
            else:                            # encode basis != decode basis
                
                # randomly decide on dl or dr in hadamard basis
                randnum = int(np.random.choice([0,1]))
                if randnum == 0:
                    dbit = "dl"
                else:
                    dbit = "dr"
    
        decoded.append(dbit)
        
    return decoded, dbase

In [10]:
def match(ebase, dbase):
    
    matchindex = []
    matchcount = 0
    
    for i in range(len(ebase)):
        if ebase[i] == dbase[i]:
            matchindex.append(i)
            matchcount += 1
    
    ematch = []
    dmatch = []  
    
    for index in matchindex:
        ematch.append(encoded[index])
        dmatch.append(decoded[index])
    
    return matchindex, matchcount, ematch, dmatch

In [11]:
def compare(ematch, dmatch, n):
    
    efinal = ematch
    dfinal = dmatch
    
    totalindex = list(range(len(ematch)))
    random.shuffle(totalindex)
    
    subsetindex = totalindex[:n]
    subsetindex.sort()
    subsetindex.reverse()
    
    esub = []
    dsub = []
    count = 0
    
    for i in subsetindex:
        esub.append(ematch.pop(i))
        dsub.append(dmatch.pop(i))
    
    for j in range(len(subsetindex)):
        if esub[j] != dsub[j]:
                count += 1
    
    error = count/len(subsetindex)
    
    detected = "no eavesdropper detected!"
    
    if abs((0.25 - error)/0.25) < 0.9:
        detected = "eavesdropper detected!"
    
    return esub, dsub, efinal, dfinal, error, detected

## 1. Error Rate (Without Eavesdropper)

from theory, bits where bases disagree are discarded, so error rate = 0.

### Mini test run

In [12]:
# alice generates (4+d)n bits and encodes them, n bits of the remaining bits are used for error checking

n = 8
d = 0
numbits = (4+d)*n

print("\n----- SIMULATION START (WITHOUT EAVESDROPPER) -----\n")

print(f"ALICE GENERATING {numbits} BITS......")
alice = np.random.choice([0, 1], size=(numbits,))
print(f"alice bits: {alice}")

print("\nALICE ENCODING BITS IN RANDOM BASIS......")
encoded, ebase = encode(alice)

print(f"encoded bits: {encoded}")

print("\nBOB DECODING BITS IN RANDOM BASIS......")

decoded, dbase = decode(encoded)

print(f"decoded bits: {decoded}")

print("\nALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......")
print(f"encoding basis: {ebase}")
print(f"decoding basis: {dbase}")

print("\nALICE AND BOB KEEP BITS IN WHICH SAME BASIS WAS USED......")
matchindex, matchcount, ematch, dmatch = match(ebase,dbase)
print(f"basis matched {matchcount}/{numbits} times.")
print(f"indexes where basis match: {matchindex}")
print(f"matched bits (encoded, {len(ematch)} bits) = {ematch}")
print(f"matched bits (decoded, {len(dmatch)} bits) = {dmatch}")

print(f"\nALICE AND BOB COMPARE A SUBSET OF {n} BITS TO CHECK ERROR RATE......")

esub, dsub, efinal, dfinal, error, detected = compare(ematch, dmatch, n)
print(f"checked bits (encoded, {len(esub)} bits) = {esub}")
print(f"checked bits (decoded, {len(dsub)} bits) = {dsub}\n")

print(f"kept bits (encoded, {len(efinal)} bits) = {efinal}")
print(f"kept bits (decoded, {len(dfinal)} bits) = {dfinal}")
print(f"error rate = {error}")
print(detected)


----- SIMULATION START (WITHOUT EAVESDROPPER) -----

ALICE GENERATING 32 BITS......
alice bits: [1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 1 0 1 0 0 0 1 1 1 0 1 0 0 0]

ALICE ENCODING BITS IN RANDOM BASIS......
encoded bits: ['dr', 'dl', 'dl', 'h', 'h', 'h', 'h', 'h', 'v', 'dr', 'h', 'h', 'h', 'dr', 'h', 'dl', 'v', 'h', 'v', 'dl', 'dr', 'h', 'dl', 'dl', 'dr', 'dr', 'v', 'h', 'v', 'dl', 'h', 'dl']

BOB DECODING BITS IN RANDOM BASIS......
decoded bits: ['dr', 'dl', 'v', 'dl', 'dl', 'h', 'dr', 'dr', 'v', 'dr', 'dl', 'h', 'h', 'dr', 'h', 'dl', 'v', 'h', 'v', 'h', 'h', 'dl', 'dl', 'h', 'dr', 'dr', 'dr', 'dr', 'v', 'dl', 'dr', 'h']

ALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......
encoding basis: [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1]
decoding basis: [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0]

ALICE AND BOB KEEP BITS IN WHICH SAME BASIS WAS USED......
basis matched 18/32 times.


### Full simulation

In [13]:
# alice generates (4+d)n bits and encodes them, n bits of the remaining bits are used for error checking

n = 1000
d = 6
numbits = (4+d)*n

print("\n----- SIMULATION START (WITHOUT EAVESDROPPER) -----\n")

print(f"ALICE GENERATING {numbits} BITS......")
alice = np.random.choice([0, 1], size=(numbits,))
#print(f"alice bits: {alice}")

print("\nALICE ENCODING BITS IN RANDOM BASIS......")
encoded, ebase = encode(alice)

#print(f"encoded bits: {encoded}")

print("\nBOB DECODING BITS IN RANDOM BASIS......")

decoded, dbase = decode(encoded)

#print(f"decoded bits: {decoded}")

print("\nALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......")
#print(f"encoding basis: {ebase}")
#print(f"decoding basis: {dbase}")

print("\nALICE AND BOB KEEP BITS IN WHICH SAME BASIS WAS USED......")
matchindex, matchcount, ematch, dmatch = match(ebase,dbase)
print(f"basis matched {matchcount}/{numbits} times.")
#print(f"indexes where basis match: {matchindex}")
#print(f"matched bits (encoded, {len(ematch)} bits) = {ematch}")
#print(f"matched bits (decoded, {len(dmatch)} bits) = {dmatch}")

print(f"\nALICE AND BOB COMPARE A SUBSET OF {n} BITS TO CHECK ERROR RATE......")

esub, dsub, efinal, dfinal, error, detected = compare(ematch, dmatch, n)
#print(f"kept bits (encoded, {len(efinal)} bits) = {efinal}")
#print(f"kept bits (decoded, {len(dfinal)} bits) = {dfinal}")
print(f"error rate = {error}")
print(detected)


----- SIMULATION START (WITHOUT EAVESDROPPER) -----

ALICE GENERATING 10000 BITS......

ALICE ENCODING BITS IN RANDOM BASIS......

BOB DECODING BITS IN RANDOM BASIS......

ALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......

ALICE AND BOB KEEP BITS IN WHICH SAME BASIS WAS USED......
basis matched 4984/10000 times.

ALICE AND BOB COMPARE A SUBSET OF 1000 BITS TO CHECK ERROR RATE......
error rate = 0.0
no eavesdropper detected!


## 2. Error Rate (With Eavesdropper)

from theory, when there is an eavesdropper, error rate is:

$$E = 0.5 \times 0.5 = 0.25$$

### Mini test run

In [17]:
# alice generates (4+d)n bits and encodes them, n bits of the remaining bits are used for error checking

n = 8
d = 0
numbits = (4+d)*n

print("\n----- SIMULATION START (WITH EAVESDROPPER) -----\n")

print(f"ALICE GENERATING {numbits} BITS......")
alice = np.random.choice([0, 1], size=(numbits,))
print(f"alice bits: {alice}")

print("\nALICE ENCODING BITS IN RANDOM BASIS......")
encoded, ebase = encode(alice)

print(f"encoded bits: {encoded}")

print("\nEVE DECODING BITS IN RANDOM BASIS......")

decoded_eve, dbase_eve = decode(encoded)
print(f"decoded bits BY EVE: {decoded_eve}")

print("\nBOB MEASURING BITS FROM EVE IN RANDOM BASIS......")

decoded, dbase = decode(decoded_eve)

print(f"decoded bits: {decoded}")

print("\nALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......")
print(f"encoding basis: {ebase}")
print(f"eve's dc basis: {dbase_eve}")
print(f"decoding basis: {dbase}")

print("\nALICE AND BOB KEEP BITS IN WHICH SAME BASIS WAS USED......")
matchindex, matchcount, ematch, dmatch = match(ebase,dbase)
print(f"basis matched {matchcount}/{numbits} times.")
print(f"indexes where basis match: {matchindex}")
print(f"matched bits (encoded, {len(ematch)} bits) = {ematch}")
print(f"matched bits (decoded, {len(dmatch)} bits) = {dmatch}")

print(f"\nALICE AND BOB COMPARE A SUBSET OF {n} BITS TO CHECK ERROR RATE......")

esub, dsub, efinal, dfinal, error, detected = compare(ematch, dmatch, n)
print(f"checked bits (encoded, {len(esub)} bits) = {esub}")
print(f"checked bits (decoded, {len(dsub)} bits) = {dsub}\n")

print(f"kept bits (encoded, {len(efinal)} bits) = {efinal}")
print(f"kept bits (decoded, {len(dfinal)} bits) = {dfinal}")
print(f"error rate = {error}")
print(detected)


----- SIMULATION START (WITH EAVESDROPPER) -----

ALICE GENERATING 32 BITS......
alice bits: [0 0 1 0 0 1 1 1 0 1 1 0 1 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 0 0 1]

ALICE ENCODING BITS IN RANDOM BASIS......
encoded bits: ['h', 'dl', 'dr', 'h', 'h', 'dr', 'v', 'dr', 'h', 'dr', 'dr', 'dl', 'dr', 'dl', 'dr', 'dl', 'dr', 'v', 'dr', 'dr', 'dl', 'dr', 'dr', 'h', 'h', 'dr', 'dr', 'v', 'v', 'dl', 'h', 'dr']

EVE DECODING BITS IN RANDOM BASIS......
decoded bits BY EVE: ['dl', 'dl', 'dr', 'dr', 'h', 'h', 'v', 'dr', 'dl', 'v', 'v', 'dl', 'v', 'h', 'dr', 'dl', 'dr', 'v', 'h', 'h', 'v', 'v', 'dr', 'dr', 'h', 'v', 'h', 'v', 'dr', 'v', 'dl', 'h']

BOB MEASURING BITS FROM EVE IN RANDOM BASIS......
decoded bits: ['dl', 'h', 'v', 'dr', 'h', 'dl', 'v', 'dr', 'dl', 'dr', 'dr', 'h', 'dr', 'dr', 'dr', 'dl', 'dr', 'dl', 'dr', 'h', 'dl', 'v', 'h', 'h', 'h', 'v', 'h', 'dl', 'h', 'dl', 'v', 'h']

ALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......
encoding basis: [0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1

### Full simulation

In [18]:
# alice generates (4+d)n bits and encodes them, n bits of the remaining bits are used for error checking

n = 1000
d = 6
numbits = (4+d)*n

print("\n----- SIMULATION START (WITH EAVESDROPPER) -----\n")

print(f"ALICE GENERATING {numbits} BITS......")
alice = np.random.choice([0, 1], size=(numbits,))
#print(f"alice bits: {alice}")

print("\nALICE ENCODING BITS IN RANDOM BASIS......")
encoded, ebase = encode(alice)

#print(f"encoded bits: {encoded}")

print("\nEVE DECODING BITS IN RANDOM BASIS......")

decoded_eve, dbase_eve = decode(encoded)
#print(f"decoded bits BY EVE: {decoded_eve}")

print("\nBOB MEASURING BITS FROM EVE IN RANDOM BASIS......")

decoded, dbase = decode(decoded_eve)

#print(f"decoded bits: {decoded}")

print("\nALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......")
#print(f"encoding basis: {ebase}")
#print(f"eve's dc basis: {dbase_eve}")
#print(f"decoding basis: {dbase}")

print("\nALICE AND BOB KEEP BITS IN WHICH SAME BASIS WAS USED......")
matchindex, matchcount, ematch, dmatch = match(ebase,dbase)
print(f"basis matched {matchcount}/{numbits} times.")
#print(f"indexes where basis match: {matchindex}")
#print(f"matched bits (encoded, {len(ematch)} bits) = {ematch}")
#print(f"matched bits (decoded, {len(dmatch)} bits) = {dmatch}")

print(f"\nALICE AND BOB COMPARE A SUBSET OF {n} BITS TO CHECK ERROR RATE......")

esub, dsub, efinal, dfinal, error, detected = compare(ematch, dmatch, n)
#print(f"kept bits (encoded, {len(efinal)} bits) = {efinal}")
#print(f"kept bits (decoded, {len(dfinal)} bits) = {dfinal}")
print(f"error rate = {error}")
print(detected)


----- SIMULATION START (WITH EAVESDROPPER) -----

ALICE GENERATING 10000 BITS......

ALICE ENCODING BITS IN RANDOM BASIS......

EVE DECODING BITS IN RANDOM BASIS......

BOB MEASURING BITS FROM EVE IN RANDOM BASIS......

ALICE AND BOB ANNOUNCING ENCODING / DECODING BASIS......

ALICE AND BOB KEEP BITS IN WHICH SAME BASIS WAS USED......
basis matched 4974/10000 times.

ALICE AND BOB COMPARE A SUBSET OF 1000 BITS TO CHECK ERROR RATE......
error rate = 0.248
eavesdropper detected!


#### Summary:
- Error rate without eavesdropper is 0 --> matches theoretical predictions
- Error rate with eavesdropper is ~0.25 --> matches theoretical predictions